In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')

import lsdb
from lsdb import lsdb_client
client = lsdb_client(dask_on_ray=False, num_workers=12)

In [4]:
client.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46057/status,
Dashboard: http://127.0.0.1:46057/status,Workers: 12
Total threads: 12,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38311,Workers: 12
Dashboard: http://127.0.0.1:46057/status,Total threads: 12
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:44369,Total threads: 1
Dashboard: http://127.0.0.1:36918/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:38095,


In [6]:
#Load ZTF, Gaia, and ZTF sources hipscats
gaia = lsdb.read_hipscat("/data3/epyc/projects3/ivoa_demo/gaia/catalog", 
                        columns=['ra', 'dec', 'bp_rp', 
                                'parallax', 'parallax_over_error', 
                                'phot_g_mean_mag', 
                                'pmra', 'pmdec', 
                                'classprob_dsc_combmod_star', 
                                'logg_gspphot',
                                'teff_gspphot'])

ztf = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14", 
                       columns=['ra', 'dec', 'ps1_objid', 
                               'nobs_g', 'nobs_r', 'nobs_i', 
                               'Npix'])

#sources load takes a minute, since it creates a healpix alignment on load
ztf_sources = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source",
                        columns=['index', 'ps1_objid',
                                       'ra', 'dec', 
                                       'catflags', 
                                       'fieldID', 
                                       'mjd', 'band', 'mag', 'magerr', 'Npix'])

In [35]:
# crossmatch ZTF + Gaia
_all_sky = gaia.cone_search(ra=25, dec=5, radius=3).crossmatch(ztf).query(
    "nobs_g_ztf_dr14 > 50 and nobs_r_ztf_dr14 > 50 and \
    parallax_gaia > 0 and parallax_over_error_gaia > 5 and \
    teff_gspphot_gaia > 5380 and teff_gspphot_gaia < 7220 and logg_gspphot_gaia > 4.5 \
    and logg_gspphot_gaia < 4.72 and classprob_dsc_combmod_star_gaia > 0.5").compute()


In [36]:
# catalog object
_sample_hips = lsdb.from_dataframe(
    _all_sky, 
    lowest_order=5, 
    highest_order=8, 
    set_hipscat_index=False, 
    ra_column="ra_gaia", 
    dec_column="dec_gaia", 
    threshold=1e10
)

# don't compute table...
ss = _sample_hips.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
)

In [37]:
print (f"Number of unique sources: {len(_all_sky.index.unique())}")

Number of unique sources: 83


In [38]:
# select only good catalog detections
good_detections_table = ss.query("catflags==0")

## TAPE

In [39]:
import dask.dataframe as dd
from tape import Ensemble, ColumnMapper

In [13]:
# Initialize an Ensemble
ens = Ensemble(client=client.client)
ens.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46057/status,
Dashboard: http://127.0.0.1:46057/status,Workers: 12
Total threads: 12,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38311,Workers: 12
Dashboard: http://127.0.0.1:46057/status,Total threads: 12
Started: 22 minutes ago,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:44369,Total threads: 1
Dashboard: http://127.0.0.1:36918/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:38095,


In [40]:
# ColumnMapper Establishes which table columns map to timeseries quantities
colmap = ColumnMapper(
        id_col='_hipscat_index',
        time_col='mjd',
        flux_col='mag',
        err_col='magerr',
        band_col='band',
      )

# We can read from parquet
ens.from_dask_dataframe(
    source_frame=good_detections_table._ddf,
    object_frame=_sample_hips._ddf,
    column_mapper=colmap,
    sync_tables=False, # Avoid doing an initial sync
    sorted=True, # If the input data is already sorted by the chosen index
    sort=False,
)

In [41]:
pth = "/astro/users/atzanida/datalrg"

In [45]:
# store good detections?!
good_detections_table._ddf.to_parquet(f"{pth}/all_sky_fg_good_detections_table.parquet")

In [ ]:
# worked! 